In [ ]:
import os
os.environ['WANDB_DISABLED']="true"

# Load Flan-T5-base

Reference Flan-T5-base [link text](https://https://arxiv.org/pdf/2210.11416)

# Install and load libraries

In [ ]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.0+cu121 requires torch==2.3.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.18.0 requires torch

In [ ]:
# install git-fls for pushing model and logs to the hugging face hub
!sudo apt-get install git-lfs --yes

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

from datasets import Dataset, load_metric

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load data

In [ ]:
# Paths to the files
path_train_short = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Data/Short_dialogue/train_short.csv'
path_test_short = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Data/Short_dialogue/test_short.csv'
path_val_short = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Data/Short_dialogue/val_short.csv'

# Load the datasets
train_short = pd.read_csv(path_train_short)
test_short = pd.read_csv(path_test_short)
val_short = pd.read_csv(path_val_short)

# Check the first few rows to ensure they're loaded correctly
print("Short_dialogue_train_data:")
print(train_short.head())

# Check the shape
print("Shape of short_dialogue_train_data:")
print(train_short.shape)
print("Shape of short_dialogue_test_data:")
print(test_short.shape)
print("Shape of short_dialogue_val_data:")
print(val_short.shape)

Short_dialogue_train_data:
  section_header                                       section_text  \
0          GENHX  The patient is a 75-year-old female who comes ...   
1      FAM/SOCHX         Significant for diabetes and hypertension.   
2  PASTMEDICALHX                  Significant for anxiety disorder.   
3          GENHX  The patient is a 77-year-old female who is una...   
4      FAM/SOCHX                                   Noncontributory.   

                                            dialogue  Dialogue_Length  \
0  Doctor: Welcome to the clinic. I am Doctor Fra...             1396   
1  Doctor: Does anyone else in your family suffer...              175   
2  Doctor: Have we gone over your survey results ...              256   
3  Guest_clinician: How old is the patient? Docto...              438   
4  Doctor: Do you have a known- Patient: Drug all...              105   

   Summary_Length  
0             677  
1              42  
2              33  
3             325  
4      

The input and target sequence lengths used in finetuning are 1024 and 256
Based on data analysis
- Min dialogue length: 6
- Ave dialogue length: 104
- Max dialogue length: 1509
- 95% dialogue length: 327
- Median dialogue length: 63
- Min summary length: 1
- Ave summary length: 39
- Max summary length: 1020
- 95% summary length: 155
- Median summary length: 14

# Experiment with original model

In [ ]:
model_name='google/flan-t5-base'

t5_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(t5_model))


trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.00%


In [ ]:
train_short = Dataset.from_pandas(train_short)
test_short = Dataset.from_pandas(test_short)
val_short = Dataset.from_pandas(val_short)


In [ ]:
## test the model

index = 10

dialogue = test_short[index]['dialogue']
summary = test_short[index]['section_text']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    t5_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

Doctor: History of any illnesses, surgeries, or hospitalizations? Patient: I dislocated my wrist when I was a kid but that's all.

Summary:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Surgeries: None. Injuries: Dislocated wrist. Illnesses: None.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
The doctor will take a blood test.


# Fine-tuning the model

## Tokenization

In [ ]:
# tokennize
max_input_length = 1024
max_output_length = 512
def tokenize_function(batch):
    start_prompt = 'Summarize the following patientdoctor dialogue. Include all medically relevant information, including family history, diagnosis, past medical and surgical history, immunizations, lab results and known allergies. You should first predict the most relevant clinical note section header and then summarize the dialogue. Dialogue:.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in batch["dialogue"]]
    batch['input_ids'] = tokenizer(prompt, padding="max_length", max_length=max_input_length, truncation=True, return_tensors="pt").input_ids
    batch['labels'] = tokenizer(batch["section_text"], padding="max_length", max_length=max_output_length, truncation=True, return_tensors="pt").input_ids
    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

In [ ]:
# map the function
batch_size = 2
train_short = train_short.map(
    tokenize_function,
    batched=True,
    batch_size=batch_size,
    remove_columns=["section_header", "section_text", "dialogue","Dialogue_Length","Summary_Length"],
)

Map:   0%|          | 0/1360 [00:00<?, ? examples/s]

In [ ]:
test_short = test_short.map(
    tokenize_function,
    batched=True,
    batch_size=batch_size,
    remove_columns=["section_header", "section_text", "dialogue","Dialogue_Length","Summary_Length"],
)

Map:   0%|          | 0/222 [00:00<?, ? examples/s]

In [ ]:
val_short = val_short.map(
    tokenize_function,
    batched=True,
    batch_size=batch_size,
    remove_columns=["section_header", "section_text", "dialogue","Dialogue_Length","Summary_Length"],
)

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

In [ ]:
# check the shape after tokenization
sample_input_id = train_short['input_ids'][0:1]
sample_label = train_short['labels'][0:1]
print("tokenized_train_exp: ", train_short)
print("\nsample_input_id: ", len(sample_input_id[0]), sample_input_id)
print("\nsample_label: ", len(sample_label[0]), sample_label)

tokenized_train_exp:  Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 1360
})

sample_input_id:  1024 [[12198, 1635, 1737, 8, 826, 1868, 26, 32, 5317, 7478, 5, 15746, 66, 1035, 120, 2193, 251, 6, 379, 384, 892, 6, 8209, 6, 657, 1035, 11, 11685, 892, 6, 256, 51, 202, 1707, 7, 6, 7690, 772, 11, 801, 18500, 5, 148, 225, 166, 9689, 8, 167, 2193, 3739, 2232, 1375, 13956, 11, 258, 21603, 8, 7478, 5, 5267, 10384, 10, 5, 7582, 10, 5242, 12, 8, 5998, 5, 27, 183, 7582, 4937, 5, 17656, 10, 1562, 25, 5, 94, 19, 1245, 12, 942, 25, 5, 7582, 10, 363, 3200, 25, 139, 217, 140, 469, 58, 17656, 10, 27, 317, 27, 164, 43, 141, 3, 9, 9529, 5, 7582, 10, 363, 2817, 12, 143, 25, 317, 24, 25, 141, 3, 9, 9529, 58, 17656, 10, 100, 1379, 6, 27, 1800, 114, 424, 16, 82, 17703, 5, 94, 47, 3, 9, 6765, 1829, 5, 37, 29, 27, 1800, 310, 1227, 19974, 5, 499, 646, 609, 11, 9657, 1800, 3, 29, 5937, 5, 7582, 10, 3963, 25, 2615, 13645, 58, 17656, 10, 27, 278, 31, 17, 214, 5, 94, 19, 3, 9, 720, 15266, 5, 27, 3, 1

## Training

In [ ]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from transformers import DataCollatorForSeq2Seq

# ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=t5_model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [ ]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from torch.optim import AdamW

# Clear any cached memory to prevent out-of-memory errors
torch.cuda.empty_cache()

# Data collator is set for dynamic padding
data_collator = DataCollatorForSeq2Seq(tokenizer, model=t5_model, return_tensors="pt")

# Define custom optimizer
optimizer = AdamW(t5_model.parameters(), lr=1e-3, weight_decay=0.01)

repository_id = 'agnesem/flan-t5'  # Hugging Face Hub repository


# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=1e-3,
    num_train_epochs=15,
    weight_decay=0.01,
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=t5_model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_short,
    eval_dataset=val_short,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, None),  # Pass the custom optimizer
)

In [ ]:
# # Start training
torch.cuda.empty_cache()
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.105961,24.881700,8.876000,22.562900,23.835400,13.605042
2,No log,2.052915,26.347100,9.213700,23.270400,24.946200,14.504202
3,1.934200,2.061843,29.704500,11.608300,25.470600,27.250100,14.966387
4,1.934200,2.094275,31.963200,13.123200,28.372200,30.039500,14.546218
5,1.934200,2.155134,34.788400,13.815500,30.668900,32.794700,13.781513
6,1.167800,2.255187,31.921600,11.360700,28.034900,29.852200,14.554622
7,1.167800,2.328453,32.636700,10.778100,27.502500,29.055700,14.621849
8,1.167800,2.451812,31.785200,11.282500,26.983800,28.927100,14.672269
9,0.820700,2.550814,32.992300,11.576600,28.574500,30.636900,14.445378
10,0.820700,2.568146,33.790900,11.893300,28.846700,30.504400,13.907563


TrainOutput(global_step=2550, training_loss=1.0051776960784313, metrics={'train_runtime': 1047.9805, 'train_samples_per_second': 19.466, 'train_steps_per_second': 2.433, 'total_flos': 2.79380999798784e+16, 'train_loss': 1.0051776960784313, 'epoch': 15.0})

In [ ]:
# # save the model
trainer.save_model("./flan_t5_base_finetuned_MTS_dialogue")
tokenizer.save_pretrained("./flan_t5_base_finetuned_MTS_dialogue")

('./flan_t5_base_finetuned_MTS_dialogue/tokenizer_config.json',
 './flan_t5_base_finetuned_MTS_dialogue/special_tokens_map.json',
 './flan_t5_base_finetuned_MTS_dialogue/spiece.model',
 './flan_t5_base_finetuned_MTS_dialogue/added_tokens.json',
 './flan_t5_base_finetuned_MTS_dialogue/tokenizer.json')

In [ ]:
# push to huggingface
from huggingface_hub import HfApi, HfFolder

model_name = "agnesem/flan_t5_base_finetuned_MTS_dialogue"
api = HfApi()

# Create the repository if it doesn't exist
api.create_repo(repo_id=model_name, repo_type="model", exist_ok=True)

# Upload the model files
api.upload_folder(
     folder_path="./flan_t5_base_finetuned_adj_MTS_dialogue",
     repo_id=model_name,
     repo_type="model"
 )

CommitInfo(commit_url='https://huggingface.co/agnesem/flan_t5_base_finetuned_MTS_dialogue/commit/4d48a875788ec1ae425bbacb3dda7947c9aa3faa', commit_message='Upload folder using huggingface_hub', commit_description='', oid='4d48a875788ec1ae425bbacb3dda7947c9aa3faa', pr_url=None, pr_revision=None, pr_num=None)

## Model evaluation

In [ ]:
# Load the fine-tuned model and tokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("agnesem/flan_t5_base_finetuned_MTS_dialogue")
model = AutoModelForSeq2SeqLM.from_pretrained("agnesem/flan_t5_base_finetuned_MTS_dialogue", torch_dtype=torch.bfloat16)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/495M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# Paths to the files
path_test_short = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Data/Short_dialogue/test_short.csv'

# Load the datasets
test_short = pd.read_csv(path_test_short)

# Check the shape
print("Shape of short_dialogue_test_data:")
print(test_short.shape)


Shape of short_dialogue_test_data:
(222, 5)


In [ ]:
sub_set_test = test_short[:5]

In [ ]:
sub_set_test = Dataset.from_pandas(sub_set_test)

In [ ]:
test_short = Dataset.from_pandas(test_short)

In [ ]:
max_input_length = 1024
max_output_length = 512
def tokenize_function(batch):
    start_prompt = 'Summarize the following patientdoctor dialogue. Include all medically relevant information, including family history, diagnosis, past medical and surgical history, immunizations, lab results and known allergies. You should first predict the most relevant clinical note section header and then summarize the dialogue. Dialogue:.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in batch["dialogue"]]
    batch['input_ids'] = tokenizer(prompt, padding="max_length", max_length=max_input_length, truncation=True, return_tensors="pt").input_ids
    batch['labels'] = tokenizer(batch["section_text"], padding="max_length", max_length=max_output_length, truncation=True, return_tensors="pt").input_ids

    return batch

In [ ]:
sub_set_test = sub_set_test.map(
    tokenize_function,
    batched=True,
    batch_size=batch_size,
    remove_columns=["section_header", "section_text", "dialogue","Dialogue_Length","Summary_Length"],
)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [ ]:
test_short = test_short.map(
    tokenize_function,
    batched=True,
    batch_size=batch_size,
    remove_columns=["section_header", "section_text", "dialogue","Dialogue_Length","Summary_Length"],
)

Map:   0%|          | 0/222 [00:00<?, ? examples/s]

In [ ]:
def model_generation(model, tokenizer, dataset):
    predictions = []
    references = []

    for example in dataset:
        input_ids = example["input_ids"]
        reference_ids = example["labels"]

        # Decode input_ids
        input_text = tokenizer.decode(input_ids, skip_special_tokens=True)
        reference_text = tokenizer.decode(reference_ids, skip_special_tokens=True)

        # Generate summary
        outputs = model.generate(input_ids=torch.tensor([input_ids]))
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Append to lists for ROUGE calculation
        predictions.append(generated_text)
        references.append(reference_text)
    return predictions, references

# Call the function with the correct argument
t5_predictions, t5_references = model_generation(model, tokenizer, test_short)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
from rouge_score import rouge_scorer

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Compute ROUGE scores for each pair of prediction and reference
rouge_scores = []
for pred, ref in zip(t5_predictions, t5_references):
    score = scorer.score(pred, ref)
    rouge_scores.append(score)


# Calculate average ROUGE scores
avg_rouge1 = sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rouge2 = sum([score['rouge2'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rougeL = sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores)

# Print average ROUGE scores
print(f"Average ROUGE-1: {avg_rouge1}")
print(f"Average ROUGE-2: {avg_rouge2}")
print(f"Average ROUGE-L: {avg_rougeL}")

Average ROUGE-1: 0.289646208502012
Average ROUGE-2: 0.12182130583241245
Average ROUGE-L: 0.25287963591815715


In [ ]:
with torch.cuda.amp.autocast():
    t5_predictions, t5_references = model_generation(model, tokenizer, test_short)

In [ ]:
from rouge_score import rouge_scorer

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Compute ROUGE scores for each pair of prediction and reference
rouge_scores = []
for pred, ref in zip(t5_predictions, t5_references):
    score = scorer.score(pred, ref)
    rouge_scores.append(score)


# Calculate average ROUGE scores
avg_rouge1 = sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rouge2 = sum([score['rouge2'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rougeL = sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores)

# Print average ROUGE scores
print(f"Average ROUGE-1: {avg_rouge1}")
print(f"Average ROUGE-2: {avg_rouge2}")
print(f"Average ROUGE-L: {avg_rougeL}")

Average ROUGE-1: 0.289646208502012
Average ROUGE-2: 0.12182130583241245
Average ROUGE-L: 0.25287963591815715


In [ ]:
data = {
    "Reference": t5_references,
    "Prediction": t5_predictions,
}
df = pd.DataFrame(data)

import io
import os

# Specify the directory to save the CSV files
output_dir = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Results'

# Ensure the directory exists
os.makedirs(output_dir, exist_ok=True)
df.to_csv(os.path.join(output_dir,"baseline_flan_t5_base_shortdialogue_predictions.csv"), index=False)

NameError: name 't5_references' is not defined

In [ ]:
print(t5_predictions)

['The patient is married. She lives in the same household. She has a primary care physician', 'No change in moles, no change in moles, no sudden weight loss. No change', 'He has no history of health problems. He has no primary care physician.', 'The patient is a 6-year-old male who is a followup for his ADHD', 'The patient has had a menstrual period of 06/03/2009. She has had an', 'Atrial fibrillation, no dizziness.', 'No sugar, no BBP, no thyroid problem, no h/o thyroid problems.', 'The patient has a history of urinary tract infection at the same time.', 'The patient is a 58-year-old female who has had a fall about', 'No swelling, swelling, or mass. No pain. No swelling. No symmetrical lump', 'Dislocated wrist when a child.', 'None.', 'None.', 'The patient is admitted for disseminated CMV infection. They are on polymy', 'The patient is a patient of the patient. She is a assistant to the patient.', 'The patient is a patient with a history of lymph node swollen for', 'No known medical p